# Final Model - SAC

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("tf.keras available:", hasattr(tf, "keras"))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.10.0
tf.keras available: True
Num GPUs Available:  1


In [2]:
from structure import DMC_structure
import numpy as np
import matplotlib.pyplot as plt
import os 

from DMC_Env import DMC_Env

import logging
from datetime import datetime

from sac import SoftActorCritic
from replay_buffer import ReplayBuffer

tf.keras.backend.set_floatx('float32')

logging.basicConfig(level='INFO')


As per the format explained above, the DMC chain is initialized below.

In [3]:
DMCarr = [[] for i in range(10)]
            # index, next, func, goal, input (T, P, Keq)
DMCarr[0] = [0, [0, 1], "DMC0", 400, [350, 5, 1]]
DMCarr[1] = [1, [2], "DMC1", 500, [350, 5, 1]]
DMCarr[2] = [2, [3, 5], "DMC2", 500, [350, 5, 1]]
DMCarr[3] = [3, [4], "DMC3", 500, [350, 5, 1]]
DMCarr[4] = [4, [5], "DMC4", 500, [350, 5, 1]]
DMCarr[5] = [5, [], "DMC5", 500, [350, 5, 1]]
DMCarr[6] = [6, [1, 6], "DMC6", 500, [350, 5, 1]]
DMCarr[7] = [7, [3, 7], "DMC7", 500, [350, 5, 1]]
DMCarr[8] = [8, [5, 8], "DMC8", 500, [350, 5, 1]]
DMCarr[9] = [9, [2, 3, 4, 9], "DMC9", 500, [350, 5, 1]]
# DMCarr[2] = [2, [], "Dummy", 0, [0, 0, 0]]

print("DMC array:", DMCarr)
struct = DMC_structure(DMCarr)

DMC array: [[0, [0, 1], 'DMC0', 400, [350, 5, 1]], [1, [2], 'DMC1', 500, [350, 5, 1]], [2, [3, 5], 'DMC2', 500, [350, 5, 1]], [3, [4], 'DMC3', 500, [350, 5, 1]], [4, [5], 'DMC4', 500, [350, 5, 1]], [5, [], 'DMC5', 500, [350, 5, 1]], [6, [1, 6], 'DMC6', 500, [350, 5, 1]], [7, [3, 7], 'DMC7', 500, [350, 5, 1]], [8, [5, 8], 'DMC8', 500, [350, 5, 1]], [9, [2, 3, 4, 9], 'DMC9', 500, [350, 5, 1]]]


Enviornment Setup

In [4]:
args = {
    'seed': 42,
    'render': False,
    'verbose': False,
    'batch_size': 128,
    'epochs': 50,
    'start_steps': 0,
    'model_path': '../data/models/',
    'model_name': f'{str(datetime.utcnow().date())}-{str(datetime.utcnow().time())}',
    'gamma': 0.99,
    'polyak': 0.995,
    'learning_rate': 0.001,
}



In [5]:
# Define DMC environment setup
env = DMC_Env(DMCarr)

state_space = env.observation_space.shape[0]
action_space = env.action_space.shape[0]

replay = ReplayBuffer(state_space, action_space)

log_dir = args['model_path'] + '/logs/' + datetime.utcnow().strftime("%Y%m%d-%H%M%S")
writer = tf.summary.create_file_writer(log_dir)

sac = SoftActorCritic(action_space, writer,
                      learning_rate=args['learning_rate'],
                      gamma=args['gamma'],
                      polyak=args['polyak'])



In [6]:
n = 10 #of episodes to run


In [8]:
episode_rewards = []
global_step = 0
episode = 0
prev_avg_episode_reward = None

# Run for n episodes
for _ in range(n):
    current_state = env.reset()
    step = 1
    episode_reward = 0
    done = False

    while not done:
        if args['render']:
            env.render()

        # Choose action: use fully random action during exploration phase.
        if global_step < args['start_steps']:
            action = env.action_space.sample()
        else:
            action = sac.sample_action(current_state)
            # In case the action is still a scalar, force it into an array.
            if np.isscalar(action):
                action = np.array([action])

        # Step in the environment.
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward
        end = 0 if done else 1

        # Optional logging per step.
        if args.get('verbose', False):
            logging.info(f"Global step: {global_step}")
            logging.info(f"Current state: {current_state}")
            logging.info(f"Action: {action}")
            logging.info(f"Reward: {reward}")
            logging.info(f"Next state: {next_state}")
            logging.info(f"End flag: {end}")

        # Store the transition in replay buffer.
        replay.store(current_state, action, reward, next_state, end)

        current_state = next_state
        step += 1
        global_step += 1

    # Training: only if enough samples are available and the exploration phase is over.
    if replay.total_size > args['batch_size'] and global_step > args['start_steps']:
        for epoch in range(args['epochs']):
            current_states, actions, rewards, next_states, ends = replay.fetch_sample(num_samples=args['batch_size'])
            critic1_loss, critic2_loss, actor_loss, alpha_loss = sac.train(
                current_states, actions, rewards, next_states, ends
            )

            if args.get('verbose', False):
                print(f"Episode {episode}, Global step {global_step}, Epoch {epoch}:",
                      critic1_loss.numpy(), critic2_loss.numpy(),
                      actor_loss.numpy(), f"Episode Reward: {episode_reward}")

            # Increase the training epoch step and update target networks each epoch.
            sac.epoch_step += 1
            sac.update_weights()  # Now uses assign() inside the SAC module.

    # Save model every 100 episodes (adjustable as needed).
    if episode % 1 == 0:
        # Generate a safe timestamp without invalid characters (no colons)
        safe_timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        args['model_name'] = safe_timestamp

        # Define the full directory where the model will be saved
        model_dir = args['model_path'] + args['model_name']

        # Create the directory if it doesn't exist
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        # Save model weights using the corrected file path
        sac.policy.save_weights(model_dir + '/model.weights.h5')

    # Update reward history and compute the average over the last 100 episodes.
    episode_rewards.append(episode_reward)
    episode += 1
    avg_episode_reward = sum(episode_rewards[-100:]) / len(episode_rewards[-100:])

    # Print the reward and average.
    print(f"Episode {episode} reward: {episode_reward}")
    print(f"Episode {episode} Average episode reward: {avg_episode_reward}")

    # Calculate and print the change in average reward compared to the previous episode.
    if prev_avg_episode_reward is not None:
        change = avg_episode_reward - prev_avg_episode_reward
        print(f"Change in average reward: {change}")
    prev_avg_episode_reward = avg_episode_reward


Action:  [ 0.05632815 -0.95496047  0.80341876  0.59817153  0.17920485  0.60808253
 -0.7119637  -0.92176706  0.8700592  -0.0315312 ]
Action:  [ 0.9984779  -0.9864634   0.9906103   0.9420792   0.82014775 -0.9360396
  0.9764237  -0.9920668  -0.9877189  -0.45587122]
Action:  [ 0.96600205 -0.9977313   0.9932906   0.9997331   0.96347374 -0.9603774
  0.9927096  -0.99925107 -0.9999338  -0.91243273]
Action:  [ 0.9930186  -0.61856     0.95793563  0.9922264   0.9684654  -0.55979717
  0.98952407 -0.9232657  -0.39732394 -0.8638626 ]
Action:  [ 0.9842989   0.10864007  0.9983389   0.9559227   0.13811572 -0.89092135
  0.70624423 -0.95884323  0.8331808  -0.7949282 ]
Action:  [ 0.97339803 -0.9861721   0.9769378   0.97335863  0.4836478  -0.5794008
  0.8536994  -0.63662136  0.1088743  -0.76746655]
Action:  [ 0.9971468  -0.9687279   0.99974084  0.9947784   0.94780165 -0.9796121
  0.98441446 -0.975485   -0.69826883 -0.564139  ]
Action:  [ 0.9385732  -0.7893838   0.99397814  0.9416856  -0.40691784 -0.5602084

KeyboardInterrupt: 

## To Do

(1) print actions taken
(2) constrain actions
(3) why is avg = reward


- why is absurdly negative? fix reward
- why is avg = reward?
- how do I actually extract the final correct policy?

Done:
- running on GPU, but push to ICE in worst case. Every loop now takes just 30 seconds.
- Reward function is updated, with minimal performance improvements. It's still absurdly negative - which indicates that the actions picked and initial conditions are terrible.
- testing script shows that the action space is correctly defined - then why is reward so bad? 

## Discussion

The program predicted that the optimal DMC set temperatures should be 897 degrees and 995 degrees. We have found that this solution is not stable, and that previous iterations have predicted other temperatures such as 500 degrees and 998 degrees. Given that our bounds are (250, 500) and (400, 600), the reward function needs more tuning in order to stay within the bounds of our system and provide a stable solution. We tried different combinations of functions, and did not get a satisfactory result - we may write a continous rather than a linear reward function to address this.

In regards to the training, the final MSE and MAE values are relatively low and stable, indicating that the model has learned a reasonable mapping from goals (inputs) to rewards (outputs).
The small gap between training and validation suggests that the model is generalizing well rather than just memorizing the training data. Monitoring both MSE and MAE provides a more complete picture of performance: MSE helps drive the training optimization, and MAE offers a straightforward “average error” interpretation.

## Summary:

1. The goal is to have an agent control multiple DMC's within a complex process to maximize some value through reinforcement learning
2. Each DMC takes in some start conditions and attempts to optimize them within their own controls, like a PID controller for temperature. The agent only changes the goal, not the behavior itself.
3. The agent wants to maximize a reward, which is a balance between increasing $K_{eq}$ as much as possible, and staying within constraints.

For the baseline model:
1. The process is two DMC's in series.
2. The reward is based on a linear positive reward for $K_{eq}$ and a fixed negative reward for each violation of bounds
3. The exploration is a naive approach of generating data at random points, from fixed start conditions.
4. Due to the naive approach, the training basically becomes supervised learning of finding rewards given our exploration batch.
5. Due to the naive approach, finding the optimal value becomes a search through the supervised model.

We found:
1. The model trained and generalized well
2. We need to adjust our reward function

In the future we hope to:
1. Optimize a far more complex process, with many more DMC's in parallel and in series, with more random fluctuations.
2. Have a more sophisiticated and balanced reward function.
3. Do an exploration based on seeking a better reward rather than the naive approach.
4. Transition from Naive learning to Q-learning, then to more advanced RL algorithms.
